In [ ]:
import chess
import chess.pgn
import chess.uci
import random

In [ ]:
def make_score(engine, i):
    fen = positions.iloc[i].fen.strip()
    engine.ucinewgame()
    board = chess.Board(fen=fen)
    move = random.choice(list(board.legal_moves))
    board.push(move)
    fen = board.fen()
    orig_fen = fen
    engine.position(board)
    engine.go(depth=DEPTH)
    
    info_handler = engine.info_handlers[0]
    score = info_handler.info["score"][1].cp
    if score is None:
        if info_handler.info["score"][1].mate != 0:
            return None
        if info_handler.info["score"][1].mate > 0:
            score = 5000
        else:
            score = -5000

    wtm = board.turn
    if not wtm: score *= -1

    pv = info_handler.info["pv"]
    if not pv.get(1): return None
    for move in pv[1]:
        board.push(move)
    fen = board.fen()
    return orig_fen, fen, score

In [ ]:
def make_engine():
    engine = chess.uci.popen_engine(STOCKFISH)
    info_handler = chess.uci.InfoHandler()
    engine.info_handlers.append(info_handler)
    return engine

In [ ]:
from multiprocessing import Queue
from multiprocessing import Manager, Process
import os

#features_test_speed = bcolz.carray(np.empty((0,8,8,14)), rootdir='../slonik_data/features_test_speed.bc', chunklen=1024, mode='w')

n = 1
step = 25000

# for num_workers in range(10,16):
data = pd.read_pickle('/ssd/slonik_data/sf_scores_12dep.pkl')
positions = pd.read_pickle('/ssd/slonik_data/positions.pkl')
   
now = time()

def save(temp_data, data):
    temp_df = pd.DataFrame(temp_data, columns=('orig_fen', 'fen', 'score'))
    data = pd.concat([data, temp_df], ignore_index=True)
    data.to_pickle('/ssd/slonik_data/sf_scores_12dep.pkl')
    return data

def do_work(proc_num, work, output):
    engine = make_engine()
    while True:
        i = work.get()
        if i is None: 
            return
        res = make_score(engine, i)
        output.put(res)

for s in tqdm(range(380001, len(positions), step)):
    num_workers = 10
    manager = Manager()
    work = manager.Queue(num_workers)
    output = Queue()
    pool = []
    for i in range(num_workers):
        p = Process(target=do_work, args=(i, work, output))
        p.start()
        pool.append(p)

    end = min(s + step, len(positions))
    num_fen = end - s
    for i in range(s, end):
        work.put(i)

    results = []
    for i in range(num_fen):
        res = output.get()
        if res is not None:
            results.append(res)

    for i in range(num_workers): 
        work.put(None)

    for p in pool:
        p.join()
        
    data = save(results, data)

    print(num_workers, "workers", len(results), "results", (time() - now), 's')

In [ ]:
# sf_scores12 = data
sf_scores12 = pd.read_pickle('/ssd/slonik_data/sf_scores_12dep.pkl')
s = np.array(sf_scores12.score,dtype='int32')
sf_scores12.score = 2 * ((s-s.min()) / (s.max()-s.min())) - 1

In [ ]:
sf_scores12.to_pickle('/ssd/slonik_data/sf_scores_12dep.pkl')